In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-31 20:14:47.326 | INFO     | SiliconDriver   - Detected 4 PCI devices
2023-10-31 20:14:47.348 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 3 device_id: 0xfaca revision: 0)
2023-10-31 20:14:47.353 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 2 device_id: 0xfaca revision: 0)
2023-10-31 20:14:47.357 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 1 device_id: 0xfaca revision: 0)
2023-10-31 20:14:47.367 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-31 20:14:47.462 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:14:47.462 | INFO     | SiliconDriver   - Disable PCIE DMA
2023-10-31 20:14:47.462 | INFO  

# Configuration

In [2]:
batch_size = 1
sequence_size = 64
num_heads = 4
head_size = 32
hidden_size = num_heads * head_size

# Initialize activations and weights using torch

In [3]:
torch_hidden_states = torch.randn((batch_size, sequence_size, hidden_size), dtype=torch.bfloat16)

torch_attention_mask = torch.zeros((1, 1, 1, sequence_size), dtype=torch.bfloat16)
torch_attention_mask[:, :, ::2, :] = -1e9

torch_query_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_query_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_key_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_key_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_value_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_value_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_output_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_output_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)

# Convert activations and weights to ttnn

In [4]:
hidden_states = ttnn.from_torch(torch_hidden_states)
attention_mask = ttnn.from_torch(torch_attention_mask)
query_weight = ttnn.from_torch(torch_query_weight)
query_bias = ttnn.from_torch(torch_query_bias)
key_weight = ttnn.from_torch(torch_key_weight)
key_bias = ttnn.from_torch(torch_key_bias)
value_weight = ttnn.from_torch(torch_value_weight)
value_bias = ttnn.from_torch(torch_value_bias)
output_weight = ttnn.from_torch(torch_output_weight)
output_bias = ttnn.from_torch(torch_output_bias)

                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | INFO     | from_torch                                         external                                          
                     Op | I

In [5]:
hidden_states = ttnn.to_device(hidden_states, device)
attention_mask = ttnn.to_device(attention_mask, device)
query_weight = ttnn.to_device(query_weight, device)
query_bias = ttnn.to_device(query_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
key_weight = ttnn.to_device(key_weight, device)
key_bias = ttnn.to_device(key_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
value_weight = ttnn.to_device(value_weight, device)
value_bias = ttnn.to_device(value_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | INFO     | to_device                                          external                                          
                     Op | I

# Write multi_head_attention using ttnn

In [6]:
def multi_head_attention(
    hidden_states,
    attention_mask,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    *,
    head_size,
):
    batch_size, sequence_size, hidden_size = hidden_states.shape
    num_heads = hidden_size // head_size

    query = hidden_states @ query_weight
    query = query + query_bias
    query = ttnn.reshape(query, (batch_size, sequence_size, num_heads, head_size))
    query = ttnn.permute(query, (0, 2, 1, 3))

    key = hidden_states @ key_weight
    key = key + key_bias
    key = ttnn.reshape(key, (batch_size, sequence_size, num_heads, head_size))
    key = ttnn.permute(key, (0, 2, 3, 1))

    value = hidden_states @ value_weight
    value = value + value_bias
    value = ttnn.reshape(value, (batch_size, sequence_size, num_heads, head_size))
    value = ttnn.permute(value, (0, 2, 1, 3))

    attention_scores = query @ key
    attention_scores = attention_scores * (1 / (head_size**0.5))
    if attention_mask is not None:
        attention_scores = attention_scores + attention_mask

    attention_probs = ttnn.softmax(attention_scores, dim=-1)

    context_layer = attention_probs @ value
    context_layer = ttnn.permute(context_layer, (0, 2, 1, 3))
    context_layer = ttnn.reshape(context_layer, (batch_size, sequence_size, hidden_size))

    self_output = context_layer @ output_weight
    self_output = self_output + output_bias

    return self_output

# Run multi_head_attention using ttnn

In [7]:
output = multi_head_attention(
    hidden_states,
    attention_mask,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)

                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Tilize                               device                                            
                     Op | INFO     | tt::tt_metal::Matmul                               device                                            
                     Op | INFO     | tt::tt_metal::TilizeWithValPadding                 device                                            
                     Op | INFO     | tt::tt_metal::EltwiseBinaryBroadcast               device                                            
                     Op | INFO     | tt::tt_metal::Untilize                             device                                            
                     Op | INFO     | from_device                                        external                                          
                     Op | I

# Explore the output

In [8]:
print("Printing ttnn tensor")
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)
output = ttnn.from_device(output)
print(output.shape)
print(output[0, :1])

print("\n\n")
print("Printing torch tensor")
torch_output = ttnn.to_torch(output)
print(torch_output.shape)
print(torch_output[0, :1])

Printing ttnn tensor
[1, 64, 128]
                     Op | INFO     | tt::tt_metal::Untilize                             device                                            
                     Op | INFO     | from_device                                        external                                          
                     Op | INFO     | to_torch                                           external                                          
Tensor([ [-0.0991211, 1.05469, -0.679688, -2.14062, 0.832031, 1.0625, 0.398438, 0.427734, 0.792969, -1.53906, -0.738281, 0.878906, -0.328125, 0.503906, 0.237305, -0.201172, -0.65625, 0.185547, -2.01562, 0.382812, -0.742188, 2.07812, 0.0245361, 0.878906, 1.14062, 0.345703, 2.20312, 1.25, 0.347656, 0.925781, 1.02344, -0.636719, 1.00781, 0.9375, 0.566406, 0.349609, -0.503906, 1.38281, 0.357422, 0.351562, -0.4375, 2.125, 0.398438, 2, 1.01562, 1.875, 0.292969, -1.33594, 1.92188, 1.07812, 0.386719, -1.26562, 1.65625, 0, -0.726562, 1.49219, -0.124512

# Close the device

In [9]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
